Selemium这个库实际上是一个自动哈测试工具，现在已经被广泛应对JavaScript渲染的页面的爬取。但是配置要装上各种驱动，大规模部署的化是个难题。
在pyppetter中，实际上背有类似Chrome浏览器类似的Chromnium浏览器在执行一些动作进行网页渲染，开发版和正式版的区别。

In [7]:
import pyppeteer
import requests
from pyquery import PyQuery as pq
'''
为了证明request无法正常请求网页内容，然后使用pyquery来解析页面中的每一个条目。
观察源码之后，我们发现每个条目class名为quote,所以这里选用了.quote这个css选择器来选择，最后输出条目
结果是0,就证明requests无法正常抓取到相关数据。因为这个页面是javascript渲染而成的
我们看到的内容都是网页加载后又执行javascript之后才形成的，因此这些条目并不原始存在于
html代码中，而requests只抓取原始的代码。
'''

'\n为了证明request无法正常请求网页内容，然后使用pyquery来解析页面中的每一个条目。\n观察源码之后，我们发现每个条目class名为quote,所以这里选用了.quote这个css选择器来选择，最后输出条目\n结果是0,就证明requests无法正常抓取到相关数据。因为这个页面是javascript渲染而成的\n我们看到的内容都是网页加载后又执行javascript之后才形成的，因此这些条目并不原始存在于\nhtml代码中，而requests只抓取原始的代码。\n'

In [8]:
url="http://quotes.toscrape.com/js"
r=requests.get(url)
doc=pq(r.text)
print("Quotes:",doc(".Quota").length)

Quotes: 0


怎么解决：
+ 分析网页源代码数据，如果数据隐藏在html的什么地方，以javascript变量的形式存在，直接提取就好了
+ 分析ajax，很多数据是经过ajax请求时候获取的，所以可以分析其接口
+ 模拟javascript渲染过程，直接抓取渲染后的结果    
而Pyppeteer和Selenium就是使用第三种方法，

In [9]:
import asyncio

In [11]:
import asyncio
from pyppeteer import launch
from pyquery import PyQuery as pq
 
async def main():
    browser = await launch()
    page = await browser.newPage()
    await page.goto('http://quotes.toscrape.com/js/')
    doc = pq(await page.content())
    print('Quotes:', doc('.quote').length)
    await browser.close()
 
asyncio.get_event_loop().run_until_complete(main())

RuntimeError: This event loop is already running

[W:pyppeteer.chromium_downloader] start chromium download.
Download may take a few minutes.
  8%|████▉                                                          | 10516480/133194757 [1:18:00<9:39:18, 3529.44it/s]

看运行结果我们就成功匹配了class为quote的条目，总数为10条，具体内容可以用pyquery查看。
pyppeteer完成了浏览器的开启，新建页面，页面加载等操作，另外，Pyppeteer里面进行了异步操作，所以需要async/await关键词来实现。launch方法会新建一个Browser对象，然后复制给browser,然后调用newPage方法相当于浏览器中新建一个选项卡，同时新建一个Page对象，Page对象调用了goto方法相当于在浏览器中输入这个url,浏览器跳转到对应的页面进行加载，加载完成之后，再调用content方法，返回当前浏览器页面的源代码，然后进一步的我们使用pyquery进行同样的解析，就可以得到JavaScript渲染的结果了。

另外，其他的一些方法如调用asyncio的get_event_loop等方法的相关操作则属于python 异步async相关的内容了。不熟悉可以了解Python的async/await的相关知识。

通过上面的代码就可以完成javascript渲染的页面的爬取了。

这个过程中，我们没有配置Chrome浏览器，没有配置浏览器驱动，免去了一些繁琐的步骤，同样达到了Selenium的效果，还实现了异步抓取。

### 模拟网页截图
下面的例子可以模拟网页截图，保存pdf,还可以执行自定义的javascript获得特定的内容。


webdriver检测问题是怎样解决的？淘宝主要通过window.navigator.webdriver来对webdriverl进行检测，我们只需要使用javascript将它设置成为false即可

In [ ]:
import asyncio   # 异步io
form pyppeteer import launch
width, height = 1366, 768
async def main():
    browser=await launch(headless=False，
                        args=[f"--window-size={width},{height}"])
    
    page= await browser.newPage()
    await page.setViewport({"width":width},"height":height)
    
    await page.goto("http://www.taobao.com")
    await asyncio.sleep(10)
    
asyncio.get_event_loop().run_until_complete(main())


In [ ]:
import asyncio 
from pyppeteer import launch
async def main():
    browser=await launch(headless=False,args=["--disable-infobars"])
    page=await browser.newPage()
    await page.goto("https://login.taobao.com/member/login.jhtml?redirectURL=https://www.taobao.com/")
    await page.evaluate('''() =>{ Object.defineProperties(navigator,{ webdriver:{ get: () => false } }) }''')
    await asyncio.sleep(10)

asyncio.get_event_loop().run_until_complete(main())
    